In [ ]:
using BitemporalPostgres
using LifeInsuranceDataModel, LifeInsuranceProduct
using SearchLight
using SearchLightPostgreSQL
using TimeZones
ENV["SEARCHLIGHT_USERNAME"] = "mf"
ENV["SEARCHLIGHT_PASSWORD"] = "mf"
SearchLight.Configuration.load() |> SearchLight.connect

SearchLight.query("DROP SCHEMA public CASCADE")
SearchLight.query("CREATE SCHEMA public")
LifeInsuranceDataModel.load_model()

Creating Tariffs

In [ ]:
using LifeInsuranceDataModel, LifeInsuranceProduct, SearchLight
import SearchLight: Serializer.serialize

tif = get_tariff_interface(Val(1))
ProfitParticipationTariff = create_tariff("Profit participation", 1, serialize(tif.parameters),serialize(tif.contract_attributes))
tif = get_tariff_interface(Val(2))
PensionTariff = create_tariff("Pension Insurance", 2, serialize(tif.parameters), serialize(tif.contract_attributes))
tif = get_tariff_interface(Val(3))
SingleLifeRiskTariff = create_tariff("Single Life Risk Insurance", 3, serialize(tif.parameters), serialize(tif.contract_attributes))
tif = get_tariff_interface(Val(4))
JointLifeRiskTariff = create_tariff("Joint Life Risk Insurance", 4, serialize(tif.parameters), serialize(tif.contract_attributes),[1,2])



In [ ]:
cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


Pension Insurance Product

In [ ]:

pensionInsurance = Product()
pr = ProductRevision(description="Pension")

pp = ProductPart()
ppr = ProductPartRevision(
    ref_tariff=PensionTariff,
    ref_role=ppRole["Main Coverage - Life"],
    description="Main Coverage - Life",
)

pp2 = ProductPart()
ppr2 = ProductPartRevision(
    ref_tariff=ProfitParticipationTariff,
    ref_role=ppRole["Profit participation"],
    description="Profit participation Life Risk",
)

w0 = Workflow(
    type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w0)
create_component!(pensionInsurance, pr, w0)
create_subcomponent!(pensionInsurance, pp, ppr, w0)
create_subcomponent!(pensionInsurance, pp2, ppr2, w0)
commit_workflow!(w0)

Single Life Risk Insurance Product

In [ ]:

singlelifeRiskInsurance = Product()
pr = ProductRevision(description="Single Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(
    ref_tariff=SingleLifeRiskTariff,
    ref_role=ppRole["Main Coverage - Life"],
    description="Main Coverage - Life",
)

pp2 = ProductPart()
ppr2 = ProductPartRevision(
    ref_tariff=ProfitParticipationTariff,
    ref_role=ppRole["Profit participation"],
    description="Profit participation Life Risk",
)

w0 = Workflow(
    type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w0)
create_component!(singlelifeRiskInsurance, pr, w0)
create_subcomponent!(singlelifeRiskInsurance, pp, ppr, w0)
create_subcomponent!(singlelifeRiskInsurance, pp2, ppr2, w0)
commit_workflow!(w0)

A Joint Life Risk Insurance Product

In [ ]:

jointlifeRiskInsurance = Product()
pr = ProductRevision(description="Joint Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(
    ref_tariff=JointLifeRiskTariff,
    ref_role=ppRole["Main Coverage - Life"],
    description="Main Coverage - Life",
)

pp2 = ProductPart()
ppr2 = ProductPartRevision(
    ref_tariff=ProfitParticipationTariff,
    ref_role=ppRole["Profit participation"],
    description="Profit participation Life Risk",
)

w0 = Workflow(
    type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w0)
create_component!(jointlifeRiskInsurance, pr, w0)
create_subcomponent!(jointlifeRiskInsurance, pp, ppr, w0)
create_subcomponent!(jointlifeRiskInsurance, pp2, ppr2, w0)
commit_workflow!(w0)

In [ ]:
prs = prsection(1, now(tz"UTC"), ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"), 0)
rolesTariffItemPartner = load_role(TariffItemPartnerRole)

create partner woman1 nonsmoker

In [ ]:
woman1 = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="woman 1",sex="f",smoker=false)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(woman1, pr, w)
commit_workflow!(w)

create partner woman2 smoker

In [ ]:
woman2 = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="woman 2",sex="f",smoker=true)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(woman2, pr, w)
commit_workflow!(w)

create partner man1 nonsmoker

In [ ]:
man1 = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="man 1",sex="m",smoker=false)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(man1, pr, w)
commit_workflow!(w)
man1

create partner man2 smoker

In [ ]:
man2 = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="man 2",sex="m",smoker=true)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(man2, pr, w)
commit_workflow!(w)

Contract woman nonsmoker pension

In [ ]:
w1 = Workflow(
    type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(
    ref_partner=woman1.id,
    ref_role=cpRole["Policy Holder"],
    description="policiyholder ref properties",
)
create_subcomponent!(c, cpr, cprr, w1)
# pi 1

PartnerRole = tiprRole["Insured Person"]
PartnerroleMap = Dict{Integer,PartnerSection}()
PartnerroleMap[PartnerRole] = psection(woman1.id.value, now(tz"UTC"), w1.tsw_validfrom, 0)

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(
    ref_product=pensionInsurance.id,
    description="from contract creation",
)
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(
    w1,
    cpi,
    pensionInsurance.id.value,
    PartnerroleMap
)
commit_workflow!(w1)

Contract man smoker woman nonsmoker joint life

In [ ]:
w1 = Workflow(
    type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(
    ref_partner=woman2.id,
    ref_role=cpRole["Policy Holder"],
    description="policyholder ref properties",
)

create_subcomponent!(c, cpr, cprr, w1)
# pi 1
PartnerroleMap = Dict{Integer,PartnerSection}()
PartnerRole = tiprRole["Insured Person"]
PartnerroleMap[PartnerRole] = psection(woman1.id.value, now(tz"UTC"), w1.tsw_validfrom, 0)
PartnerRole = tiprRole["2nd Insured Person"]
PartnerroleMap[PartnerRole] = psection(man1.id.value, now(tz"UTC"), w1.tsw_validfrom, 0)


cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(
    ref_product=jointlifeRiskInsurance.id.value,
    description="from contract creation",
)
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(
    w1,
    cpi,
    jointlifeRiskInsurance.id.value,
    PartnerroleMap,
)
commit_workflow!(w1)

contract man smoker single life risk

In [ ]:
w1 = Workflow(
    type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(
    ref_partner=man2.id,
    ref_role=cpRole["Policy Holder"],
    description="policiyholder ref properties",
)
create_subcomponent!(c, cpr, cprr, w1)
# pi 1

PartnerRole = tiprRole["Insured Person"]
PartnerroleMap = Dict{Integer,PartnerSection}()
PartnerroleMap[PartnerRole] = psection(man2.id.value, now(tz"UTC"), w1.tsw_validfrom, 0)

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(
    ref_product=singlelifeRiskInsurance.id,
    description="from contract creation",
)
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(
    w1,
    cpi,
    singlelifeRiskInsurance.id.value,
    PartnerroleMap
)
commit_workflow!(w1)

calculation pension insurance net preminum

In [ ]:
cid = 1
h = find(Contract, SQLWhereExpression("id =?", cid))[1].ref_history
vi = find(ValidityInterval, SQLWhereExpression("ref_history=?", h), order=["ValidityInterval.id"])[1];
txntime = vi.tsdb_validfrom
reftime = vi.tsworld_validfrom

cs = csection(cid, txntime, reftime)

ti = cs.product_items[1].tariff_items[1]


In [ ]:
using JSON, LifeInsuranceProduct,MortalityTables,LifeContingencies,Yields
import LifeContingencies: ä, A
tariffparms = JSON.parse(ti.tariff_ref.ref.revision.parameters)
mts = tariffparms["mortality_tables"]
interface_id = ti.tariff_ref.ref.revision.interface_id
parms = get_tariff_interface(Val(interface_id)).calls
args=parms["calculation_target"]
args["selected"] = "net premium"
args["net premium"]["pension rate"]["value"] = "500"
args["net premium"]["m"]["value"] = "10"
args["net premium"]["n"]["value"] = "20"
args["net premium"]["frequency"]["value"] = "2"
args["net premium"]["begin"]["value"] = string(Date("2023-04-01"))
parms
parms["calculation_target"]
calculate!(Val(interface_id),ti,parms)
parms["result"]

In [ ]:
tariffparms = JSON.parse(ti.tariff_ref.ref.revision.parameters)
println(tariffparms)
interface_id = ti.tariff_ref.ref.revision.interface_id
mts = tariffparms["mortality_tables"]
i = tariffparms["interest rate"]
tgt = get_tariff_interface(Val(interface_id)).calls["calculation_target"]
tgt["selected"] = "net premium"
parms = tgt[tgt["selected"]]
parms["m"] = 10
parms["n"] = 20
parms["frequency"] = 2
parms["begin"] = string(Date("2023-04-01"))


# accessiong partner LifeInsuranceDataModel
dob = ti.partner_refs[1].ref.revision.date_of_birth
smoker = ti.partner_refs[1].ref.revision.smoker ? "smoker" : "nonsmoker"
sex = ti.partner_refs[1].ref.revision.sex
# accessing tariff data

issue_age = TariffUtilities.insurance_age(dob, Date(parms["begin"]))

life = SingleLife(
    mortality=MortalityTables.table(mts[sex][smoker]).select[issue_age])

yield = Yields.Constant(i)      # Using a flat

lc = LifeContingency(life,yield)  # LifeContingenc


r1 = ä(lc)
r2 = ä(lc,1)
r3 = ä(lc, 2)
r4 = ä(lc, 30)

println("r1")
println(r1)
println("r2")
println(r2)
println("r3")
println(r3)
println("r4")
println(r4)
r4

In [ ]:
cid = 2
h = find(Contract, SQLWhereExpression("id =?", cid))[1].ref_history
vi = find(ValidityInterval, SQLWhereExpression("ref_history=?", h), order=["ValidityInterval.id"])[1];
txntime = vi.tsdb_validfrom
reftime = vi.tsworld_validfrom

cs = csection(cid, txntime, reftime)

ti = cs.product_items[1].tariff_items[1]

tariffparms = JSON.parse(ti.tariff_ref.ref.revision.parameters)
println(tariffparms)
mts = tariffparms["mortality_tables"]
i = tariffparms["interest rate"]
interface_id = ti.tariff_ref.ref.revision.interface_id
tgt = get_tariff_interface(Val(interface_id)).calls["calculation_target"]
tgt["selected"] = "net premium"
parms = tgt[tgt["selected"]]
parms["m"] = 10
parms["n"] = 20
parms["frequency"] = 2
parms["begin"] = string(Date("2023-04-01"))

# accessiong partner LifeInsuranceDataModel
dob1 = ti.partner_refs[1].ref.revision.date_of_birth
smoker1 = ti.partner_refs[1].ref.revision.smoker ? "smoker" : "nonsmoker"
sex1 = ti.partner_refs[1].ref.revision.sex
issue_age1 = TariffUtilities.insurance_age(dob1, Date(parms["begin"]))

dob2 = ti.partner_refs[2].ref.revision.date_of_birth
smoker2 = ti.partner_refs[2].ref.revision.smoker ? "smoker" : "nonsmoker"
sex2 = ti.partner_refs[2].ref.revision.sex
issue_age2 = TariffUtilities.insurance_age(dob2, Date(parms["begin"]))

# accessing tariff data


life1 = SingleLife(
    mortality=MortalityTables.table(mts[sex1][smoker1]).select[issue_age1])
life2 = SingleLife(
    mortality=MortalityTables.table(mts[sex2][smoker2]).select[issue_age2])
 jl= JointLife(lives=(life1,life2))

yield = Yields.Constant(i)      # Using a flat

lc = LifeContingency(jl, yield)  # LifeContingenc

n=30
c=5000
r1 = A(lc,30)
r2=ä(lc,n)
5000 * A(lc, 30) / ä(lc, 30, frequency=1n)
A(lc,30)#
 #/ ä(lc, 30,frequency=1n)
 i